### Hooks

#### The State Hook

##### Objects in State
We can also use state with objects. When we work with a set of related variables, it can be very helpful to group them into an object. Let’s look at an example of this in action.
```
export default function Login() {
  const [formState, setFormState] = useState({});
  const handleChange = ({ target }) => {
    const { name, value } = target;
    setFormState((prev) => ({
      ...prev,
      [name]: value
    }));
  };
 
  return (
    <form>
      <input
        value={formState.firstName}
        onChange={handleChange}
        name="firstName"
        type="text"
      />
      <input
        value={formState.password}
        onChange={handleChange}
        type="password"
        name="password"
      />
    </form>
  );
}
```

A few things to notice:

- We use a state setter callback function to update a state based on the previous value.

- The spread syntax is the same for objects as for arrays: `{ ...oldObject, newKey: newValue }`.

- We reuse our event handler across multiple inputs by using the input tag’s `name` attribute to identify which input the change event came from.

Once again, when updating the state with `setFormState()` inside a function component, we do not modify the same object. We must copy over the values from the previous object when setting the next value of a state. Thankfully, the spread syntax makes this super easy to do!

Anytime one of the input values is updated, the `handleChange()` function will be called. Inside this event handler, we use object destructuring to unpack the `target` property from our `event` object, then we use object destructuring again to unpack the `name` and `value` properties from the `target` object.

Inside our state setter callback function, we wrap our curly brackets in parentheses like so:
```
setFormState((prev) => ({ ...prev }))
```

This tells JavaScript that our curly brackets refer to a new object to be returned. We use `...`, the spread operator, to fill in the corresponding fields from our previous state. Finally, we overwrite the appropriate key with its updated value.

Did you notice the square brackets around the `name`? This Computed Property Name allows us to use the string value stored by the `name` variable as a property key.


1. We’ll use objects with states to build an input form.

The local state variable profile and state setter function setProfile are responsible for keeping track of the input values from our users. In our JSX, we are looking up properties stored in the profile object. This throws an error at our first render because we are attempting to get the value of a property from an object that has not been defined yet.

To fix this, initialize profile as an empty object.


2. You should now see the form rendered, but nothing will happen when we type in the input boxes. Our form does not re-render to show the keystrokes yet.

To fix this, add the onChange event listener to our JSX tags to call handleChange() whenever a user types in an input field. This way, we can determine what happens when the user changes the input by typing in the form.


3. Let’s make our handleChange() function a bit easier to read. Use object destructuring to initialize name and value in a more concise way.


4. There’s a bug in our code! Have you noticed it? Try typing in one input, then type in a different input. What happens? Why?

Each time that we call setProfile() in our event handler, we give profile the value of a new object with the name and value of the input that most recently changed, but we lose the values that were stored for inputs with any other name.

Use the spread operator to fix this bug. We want to copy over all of the values from our previous profile object whenever we call our state setter function. Use prevProfile as the argument for our state setter callback function.


5. Finally, add an event listener to the <form> tag to call our handleSubmit() function when the user submits the form.

#+name: OriginalEditProfile.js
#+begin_src javascript
import React, { useState } from "react";

export default function EditProfile() {
  const [profile, setProfile] = useState();

  const handleChange = ({ target }) => {
    const name = target.name;
    const value = target.value;
    setProfile({
      [name]: value
    });
  };

  const handleSubmit = (event) => {
    event.preventDefault();
    alert(JSON.stringify(profile, '', 2));
  };

  return (
    <form>
      <input
        value={profile.firstName || ''}
        name="firstName"
        type="text"
        placeholder="First Name"
      />
      <input
        value={profile.lastName || ''}
        type="text"
        name="lastName"
        placeholder="Last Name"
      />
      <input
        value={profile.bday || ''}
        type="date"
        name="bday"
      />
      <input
        value={profile.password || ''}
        type="password"
        name="password"
        placeholder="Password"
      />
      <button type="submit">Submit</button>
    </form>
    
  );
}
#+end_src


#+name: EditProfile.js
#+begin_src javascript
import React, { useState } from "react";

export default function EditProfile() {
  const [profile, setProfile] = useState({});

  const handleChange = ({ target }) => {
    // Use object destructuring to initialize name and value
    const {name, value} = target;

    setProfile((prevProfile) => ({
      // Use the spread operator here (...provProfile)
      ...prevProfile,
      [name]: value
    }));
  };  

  const handleSubmit = (event) => {
    event.preventDefault();
    alert(JSON.stringify(profile, '', 2));
  };

  return (
    <form>
      <input
        value={profile.firstName || ''}
        name="firstName"
        type="text"
        placeholder="First Name"
        onChange={handleChange}
      />
      <input
        value={profile.lastName || ''}
        type="text"
        name="lastName"
        placeholder="Last Name"
        onChange={handleChange}
      />
      <input
        value={profile.bday || ''}
        type="date"
        name="bday"
        onChange={handleChange}
      />
      <input
        value={profile.password || ''}
        type="password"
        name="password"
        placeholder="Password"
        onChange={handleChange}
      />
      <button type="submit" onClick={handleSubmit}>Submit</button>
    </form>
    
  );
}
#+end_src

#+name: Index.js
#+begin_src javascript
import React from 'react';
import ReactDOM from 'react-dom/client';

import App from './App.js';

ReactDOM.createRoot( 
  document.querySelector('#app')
).render(<App />)
#+end_src

#### The Effect Hook
##### Why Use useEffect?
Before Hooks, function components were only used to accept data in the form of props and return some JSX to be rendered. However, as we learned in the last lesson, the State Hook allows us to manage dynamic data, in the form of component state, within our function components.

In this lesson, we’ll use the **Effect Hook** to run some JavaScript code after each render to:

- fetch data from a back-end service.
- subscribe to a stream of data.
- manage timers and intervals.
- read from and make changes to the DOM.

Components will re-render multiple times throughout their lifetime. These key moments present the perfect opportunity to execute these “side effects”.

There are three key moments when the Effect Hook can be utilized:

1. When the component is first added, or *mounted*, to the DOM and renders.
2. When the state or props change, causing the component to re-render.
3. When the component is removed, or *unmounted*, from the DOM.

Later on in this lesson, we’ll learn how to further fine-tune exactly when the Effect Hook executes.

##### Function Component Effects
The Effect Hook tells our component to do something every time it’s rendered (or re-rendered). Combined with states, we can use the Effect Hook to create interesting dynamic changes in our web pages!

Suppose we want to allow a user to change the title of the web page tab every time they type. We can implement this with the Effect Hook (useEffect()) like so:
```
import React, { useState, useEffect } from 'react';
 
function PageTitle() {
  const [name, setName] = useState('');
 
  useEffect(() => {
    document.title = `Hi, ${name}`;
  });
 
  return (
    <div>
      <p>Use the input field below to rename this page!</p>
      <input onChange={({target}) => setName(target.value)} value={name} type='text' />
    </div>
  );
}
```

Let’s take a look at the above example in more detail. First, we import the Effect Hook from the `'react'` library:

```
import { useEffect } from 'react';
```

The `useEffect()` function has no return value as the Effect Hook is used to call another function. We pass the callback function, or *effect*, to run after a component renders as the argument of the `useEffect()` function. In our example, the following effect runs after each time the `PageTitle` component renders:

```
() => { document.title = `Hi, ${name}`;}
```

Here, we assign `Hi, ${name}` as the value of `document.title`.

The `onChange` event listener triggers the `PageTitle` component to be re-rendered every time the user types in the input. Consequently, this triggers `useEffect()` and changes the document’s title.

Notice how we use the current state inside of our effect. Even though our effect is called after the component renders, we still have access to the variables in the scope of our function component! When React renders our component, it will update the DOM as usual, and then run our effect after the DOM has been updated. This happens for every render, including the first and last one.


1. Import the Effect Hook, the State Hook, and React from the 'react' library.

Make sure to import everything in one line.


2. Call useEffect() with a callback function that creates an alert with the current value of count.

Start clicking the button to see when our alert() function is called and be sure that it is logging the values that we’d expect!


3. Use a template literal so that the message in our alert dialog reads: “Count: 0”, then “Count: 1”, then “Count: 2”, etc.

#+name: Counter.js
#+begin_src
import React, { useState, useEffect } from 'react';

export default function Counter() {
  const [count, setCount] = useState(0);
  useEffect(() => {
    alert(`Count: ${count}`);
  })
  const handleClick = () => {
    setCount((prevCount) =>  prevCount + 1);
  };

  return (
    <div>
      <p>You clicked {count} times</p>
      <button onClick={handleClick}>
        Click me
      </button>
    </div>
  );
}
#+end_src

##### Clean Up Effects
Some effects require **cleanup**. For example, we might want to add event listeners to some element in the DOM, beyond the JSX in our component. When we add event listeners to the DOM, it is important to remove those event listeners when we are done with them to avoid memory leaks!

Let’s consider the following effect:

```
useEffect(()=>{
  document.addEventListener('keydown', handleKeyPress);
  // Specify how to clean up after the effect:
  return () => {
    document.removeEventListener('keydown', handleKeyPress);
  };
})
```

If our effect didn’t return a *cleanup function*, a new event listener would be added to the DOM’s `document` object every time that our component re-renders. Not only would this cause bugs, but it could cause our application performance to diminish and maybe even crash!

Because effects run after every render and not just once, React calls our cleanup function before each re-render and before unmounting to clean up each effect call.

If our effect returns a function, then the `useEffect()` Hook always treats that as the cleanup function. React will call this cleanup function before the component re-renders or unmounts. Since this cleanup function is optional, it is our responsibility to return a cleanup function from our effect when our effect code could create memory leaks.


1. Let’s create a program that documents how many times you’ve clicked on the page.

Write an event handler named increment() that will be responsible for tracking how many times a user has clicked. Define this function so that it calls setClickCount() with a state setter callback function, adding 1 to the previous value of clickCount.


2. Import the useEffect() hook and call it with an effect that adds an event listener for 'mousedown' events on the document object. When a 'mousedown' event occurs anywhere on the document, we want our increment() event handler to be called.


3. If you haven’t already, run our code and click around the browser window. What is happening? Why is this happening?

Each time that our component renders, our effect is called, adding another event listener. With just a few clicks and rerenders, we have attached a lot of event listeners to the DOM! We need to clean up after ourselves!

Update our effect so that it returns a cleanup function that will remove our last event listener from the DOM.

In [ ]:
// Counter.js
import React, { useState, useEffect } from 'react';

export default function Counter() {
  const [clickCount, setClickCount] = useState(0);

  // your code here
  // When a 'mousedown' event occurs anywhere on the document, we want our increment() event handler to be called
  useEffect(() => {
    document.addEventListener('mousedown', increment);

    // return a cleanup function that will remove our last event listener from the DOM
    return () => {
      document.removeEventListener('mousedown', increment);
    };
  });

  // an event handler named increment() that will be responsible for tracking how many times a user has clicked. 
  // Define this function so that it calls setClickCount() with a state setter callback function,
  // adding 1 to the previous value of clickCount
  const increment = () => {
  setClickCount((prevClickCount) => prevClickCount + 1)
};

  return (
      <h1>Document Clicks: {clickCount}</h1>
  );
}

##### Control When Effects Are Called
The `useEffect()` function calls its first argument (the effect) after each time a component renders. We’ve learned how to return a cleanup function so that we don’t create performance issues and other bugs, but sometimes we want to skip calling our effect on re-renders altogether.

It is common, when defining function components, to run an effect only when the component mounts (renders the first time), but not when the component re-renders. The Effect Hook makes this very easy for us to do! If we want to only call our effect after the first render, we pass an empty array to `useEffect()` as the second argument. This second argument is called the **dependency array**.

The dependency array is used to tell the `useEffect()` method when to call our effect and when to skip it. Our effect is always called after the first render but only called again if something in our dependency array has changed values between renders.

We will continue to learn more about this second argument over the next few exercises, but for now, we’ll focus on using an empty dependency array to call an effect when a component first mounts, and if a cleanup function is returned by our effect, calling that when the component unmounts.
```
useEffect(() => {
  alert("component rendered for the first time");
  return () => {
    alert("component is being removed from the DOM");
  };
}, []); 
```
Without passing an empty array as the second argument to the `useEffect()` above, those alerts would be displayed before and after every render of our component, which is clearly not when those messages are meant to be displayed. Simply passing `[]` to the `useEffect()` function is enough to configure when the effect and cleanup functions are called!



1. Let’s get started by using the following four functions to advance the number stored in time each second:

- useEffect(): the Effect Hook, imported from the ‘react’ library.
- JavaScript setInterval() function.
- setTime(): our state setter function.
- A state setter callback function: used by setTime() to calculate the next value of time based on the previous value of time.
- Add an effect that uses the setInterval() function to call setTime() every second (or 1000 ms).


2. Our time value is updating way too quickly because the Effect Hook calls our effect after every render! Our effect is creating a new interval that updates the value of time each second. We keep adding more and more intervals that keep updating the same time variable. We need to clean up our old intervals before adding new ones!

Let’s start by creating a variable, intervalId and assign it to our setInterval() code from the previous step.

Then, below your intervalId declaration, use the return keyword to return a cleanup function. Our cleanup function should use the clearInterval() function.


3. That seems to have solved our way-too-many-intervals-all-updating-the-same-variable bug!

Let’s add an extra variable to our timer and allow the user to type a message while the timer is counting up.

First, let’s create a state variable called name with a state setter called setName() to manage the value of the input box. Set the state variable to the initial value of an empty string.


4. Great! Let’s go ahead and put that input tag in.

Add an <input> element to our JSX. Set its value attribute to our state variable name.


5. Next, define an event handler function named handleChange(). This event handler will take in the value of the user’s input and update the state variable.

handleChange() should use object destructuring on its parameter to take in target and use the state setter setName to set the value of name to target.value.


6. Excellent job! Put that handleChange() event handler to work!

Add the onChange event listener to the input tag, setting it to handleChange().

Try typing now!


7. Uh oh. More bugs. Did you notice it yet? Type your full name in the text input field. See how the timer seems to stop counting while you are typing? That’s not what we want!

What is going on here? We are creating a new interval after each render, that interval will call our state setter to update time exactly one second after each render. When we type in the input field, our component keeps re-rendering, cleaning up old intervals, and starting new ones… but our state setter never gets called until one second after we are done typing!

Let’s fix this once and for all! We really want to use a single interval. We want that interval to start ticking away after our first render and we want it to be cleaned up after the final render.

To accomplish this, use an empty dependency array!

In [ ]:
// OriginalTimer.js
import React, { useState } from 'react';

export default function Timer() {
  const [time, setTime] = useState(0);

  return (
    <>
      <h1>Time: {time}</h1>
    </>
  );
}

In [ ]:
// Timer.js
import React, { useState, useEffect } from 'react';

export default function Timer() {
  const [time, setTime] = useState(0);

  // create a state variable called name
  const [name, setName] = useState('');

  useEffect(() => {
    // create a variable intervalId and assign to setInterval() code
    const intervalId = setInterval(() => {
      setTime((prev) => prev + 1);
    }, 1000);

    // return a cleanup function
    return () => {
      clearInterval(intervalId);
    };
  }, []);

  // event handler to take in user input and update the state variable
  const handleChange = ({target}) => {
    setName(target.value);
  };


  return (
    <>
      <h1>Time: {time}</h1>
      <input value={name} onChange={handleChange} type='text' />
    </>
  );
}

##### Fetch Data
When building software, we often start with default behaviors and then modify them to improve performance.

We’ve learned that the default behavior of the Effect Hook is to call the effect function after every single render.

Next, we learned that we can pass an empty array as the second argument for `useEffect()` if we only want our effect to be called after the component’s first render.

In this exercise, we’ll learn to use the dependency array to further configure exactly when we want our effect to be called!

When our effect is responsible for fetching data from a server, we pay extra close attention to when our effect is called. Unnecessary round trips back and forth between our React components and the server can be costly in terms of:

- Processing
- Performance
- Data usage for mobile users
- API service fees

When the data that our components need to render doesn’t change, we can pass an empty dependency array so that the data is fetched after the first render. When the response is received from the server, we can use a state setter from the State Hook to store the data from the server’s response in our local component state for future renders. Using the State Hook and the Effect Hook together in this way is a powerful pattern that saves our components from unnecessarily fetching new data after every render!

An empty dependency array signals to the Effect Hook that our effect never needs to be re-run, that it doesn’t depend on anything. Specifying zero dependencies means that the result of running that effect won’t change and calling our effect once is enough.

A dependency array that is not empty signals to the Effect Hook that it can skip calling our effect after re-renders unless the value of one of the variables in our dependency array has changed. If the value of a dependency has changed, then the Effect Hook will call our effect again!

Here’s a nice example from the official React docs:
```
useEffect(() => {
  document.title = `You clicked ${count} times`;
}, [count]); // Only re-run the effect if the value stored by count changes
```


``` {javascript OriginalForecast.js}
import React, { useState, useEffect } from "react";
import { get } from './mockBackend/fetch';

export default function Forecast() {
  const [data, setData] = useState();
  const [notes, setNotes] = useState({});
  const [forecastType, setForecastType] = useState('/daily');

  useEffect(() => {
    alert('Requested data from server...');
    get('/daily').then((response) => {
      alert('Response: ' + JSON.stringify(response,'',2));
    });
  });

  const handleChange = (index) => ({ target }) =>
    setNotes((prev) => ({
      ...prev,
      [index]: target.value
    }));

  return (
    <div className='App'>
      <h1>My Weather Planner</h1>
      <div>
        <button onClick={() => setForecastType('/daily')}>5-day</button>
        <button onClick={() => setForecastType('/hourly')}>Today</button>
      </div>
      <table>
        <thead>
          <tr>
            <th>Summary</th>
            <th>Avg Temp</th>
            <th>Precip</th>
            <th>Notes</th>
          </tr>
        </thead>
        <tbody>
          {data.map((item, i) => (
            <tr key={item.id}>
              <td>{item.summary}</td>
              <td> {item.temp.avg}°F</td>
              <td>{item.precip}%</td>
              <td>
                <input
                  value={notes[item.id] || ''}
                  onChange={handleChange(item.id)}
                />
              </td>
            </tr>
          ))}
        </tbody>
      </table>
    </div>
  );
}
```


1. We’ve started building a weather planner app that will fetch data about the weather and allow our users to write some notes next to the forecast. A lot of good code has already been written, but there currently isn’t anything rendering to the screen.

Let’s read through the code and start to wrap our heads around what is going on here. What part of our code do we think is keeping the component from rendering?

In our JSX, we are trying to map over an array stored by the data state variable, but our effect that fetches this data doesn’t get called until after the first render. So during the first render, data is undefined and attempting to call map() on undefined is causing our error!

Let’s prevent this error by checking to see if the data has loaded yet. If it hasn’t, then we want our function component to just return a paragraph tag with the text “Loading…”. If the data is no longer undefined, then the data has been loaded, and we can go ahead and render the full JSX!


2. Our data fetching is being done in our effect. Notice how we are currently just using alert() messages to keep track of requesting and receiving data from our server. Instead of just stringifying the response data and showing it in an alert message, let’s store that data in our state.

When the data has been fetched, use our state setter function to store that data in our component’s state!

Remember that we want to store an array in our data state variable, not the whole response object.


3. Type in each of the notes’ input fields in our table. What do you notice? Why do you think this is happening?

Each time that we type in an input field, the component re-renders to show the new value of that field. Even though we don’t need any new data from the backend, our component is fetching new data after every render!

Use an empty dependency array to ensure that data is only fetched after our component’s first render.


4. Wow, that small code change made a huge difference in the performance of our weather planner app!

Let’s make one more improvement. Did you notice the buttons at the top of our app? We want our users to be able to choose between planning around daily weather forecasts and weekly weather forecasts. Clicking on these buttons currently doesn’t change anything. Let’s fix that!

The server has two different endpoints called: /daily and /hourly. Let’s use the value of the forecastType state variable to determine which endpoint our effect should request data from.

After making this change, our effect behaves differently based on the value of forecastType. You could say that how we use our effect depends on it! Add this variable to our dependency array so that the effect is called again, updating data appropriately, after re-renders where the user has selected a different forecast type.

In [ ]:
// Forecast.js
import React, { useState, useEffect } from "react";
import { get } from './mockBackend/fetch';

export default function Forecast() {
  const [data, setData] = useState();
  const [notes, setNotes] = useState({});
  const [forecastType, setForecastType] = useState('/daily');

  // store response data in the "data" state variable
  useEffect(() => {
    alert('Requested data from server...');
    get(forecastType).then((response) => {
      alert('Response: ' + JSON.stringify(response,'',2));
      setData(response.data);
    });
  }, [forecastType]);

  const handleChange = (index) => ({ target }) =>
    setNotes((prev) => ({
      ...prev,
      [index]: target.value
    }));

  // check if data has loaded. If not, return "Loading..."
  if (!data) {
    return <p>Loading...</p>;
  }

  return (
    <div className='App'>
      <h1>My Weather Planner</h1>
      <div>
        <button onClick={() => setForecastType('/daily')}>5-day</button>
        <button onClick={() => setForecastType('/hourly')}>Today</button>
      </div>
      <table>
        <thead>
          <tr>
            <th>Summary</th>
            <th>Avg Temp</th>
            <th>Precip</th>
            <th>Notes</th>
          </tr>
        </thead>
        <tbody>
          {data.map((item, i) => (
            <tr key={item.id}>
              <td>{item.summary}</td>
              <td> {item.temp.avg}°F</td>
              <td>{item.precip}%</td>
              <td>
                <input
                  value={notes[item.id] || ''}
                  onChange={handleChange(item.id)}
                />
              </td>
            </tr>
          ))}
        </tbody>
      </table>
    </div>
  );
}

##### Rules of Hooks
There are two main rules to keep in mind when using Hooks:

1. Only call Hooks at the top level.
2. Only call Hooks from React functions.

As we have been practicing with the State Hook and the Effect Hook, we’ve been following these rules with ease, but it is helpful to keep these two rules in mind as you take your new understanding of Hooks out into the wild and begin using more Hooks in your React applications.

When React builds the Virtual DOM, the library calls the functions that define our components over and over again as the user interacts with the user interface. React keeps track of the data and functions that we are managing with Hooks based on their order in the function component’s definition. For this reason, we always call our Hooks at the top level; we never call hooks inside of loops, conditions, or nested functions.

Instead of confusing React with code like this:

if (userName !== '') {
  useEffect(() => {
    localStorage.setItem('savedUserName', userName);
  });
}


We can accomplish the same goal while consistently calling our Hook every time:

useEffect(() => {
  if (userName !== '') {
    localStorage.setItem('savedUserName', userName);
  }
});


Secondly, Hooks can only be used in React Functions. We’ve been working with `useState()` and `useEffect()` in *function* components, and this is the most common use. The only other place where Hooks can be used is within custom hooks. Custom Hooks are incredibly useful for organizing and reusing stateful logic between function components.

In [ ]:
// {javascript OriginalShop.js}
import React, { useState, useEffect } from 'react';
import { get } from './mockBackend/fetch';

export default function Shop() {
  const [categories, setCategories] = useState();
  if (categories) {
    const [selectedCategory, setSelectedCategory] = useState();
    const [items, setItems] = useState({});
  }

  if (!categories) {
    useEffect(() => {
      get('/categories').then((response) => {
        setCategories(response.data);
      });
    });
  }

  // if (selectedCategory && !items[selectedCategory]) {
  //   useEffect(() => {
  //     get(`/items?category=${selectedCategory}`).then((response) => {
  //       setItems((prev) => ({ ...prev, [selectedCategory]: response.data }));
  //     });
  //   });
  // }

  if (!categories) {
    return <p>Loading..</p>;
  }

  return (
    <div className='App'>
      <h1>Clothes 'n Things</h1>
      <nav>
        {categories.map((category) => (
          <button key={category} onClick={() => setSelectedCategory(category)}>
            {category}
          </button>
        ))}
      </nav>
      <h2>{selectedCategory}</h2>
      <ul>
        {!items[selectedCategory]
          ? null
          : items[selectedCategory].map((item) => <li key={item}>{item}</li>)}
      </ul>
    </div>
  );
}

1. The code that we are starting with has a lot of good ideas, but there are some bugs that we need to help sort out. Let’s get started by refactoring the code so that the State Hook is always called at the top level.

It looks like the developers that wrote this code wanted to hold off on using the `selectedCategory` and `items` state variables until after the `categories` have been fetched. Conceptually this makes sense, but React requires that all hooks be called on every render, so nesting these `useState()` calls is not a valid option.

First, remove the `if (categories)` statement, and the surrounding curly braces `{ }` to bring all of our State Hook calls to the top level.


2. Next, to be clear about initial values, let’s explicitly set the initial state value for `categories` and `selectedCategory` to `null`.


3. It looks like the idea behind using this expression: `if (!categories)` was to only fetch the categories data from the server once. Nesting a call to the Effect Hook inside of a condition like this will cause different hooks to be called on different re-renders, resulting in errors. Luckily, we know a better way!

Refactor this code so that the effect responsible for fetching the `categories` data from the backend and saving it to local state follows the rules for Hooks and only fetches the categories data once.


4. Whew, we’re making great progress! It’s such a nice feeling to turn error screens into working code, isn’t it?

Now that we are fetching the list of categories from the backend and successfully rendering buttons for each of these to the screen, we are ready to use another effect to fetch the items for each of these categories, when the user clicks on each of them!

Uncomment the block of code that was attempting to do this, and refactor it so that we follow the rules of Hooks. To optimize performance, only call the backend for data when we don’t yet have it stored in the component’s state like this code was trying to do.

In [ ]:
// {javascript Shop.js}
import React, { useState, useEffect } from 'react';
import { get } from './mockBackend/fetch';

export default function Shop() {
  // explicitly set the initial state value for categories and selectedCategory to null
  const [categories, setCategories] = useState(null);

  const [selectedCategory, setSelectedCategory] = useState(null);
  const [items, setItems] = useState({});

  /*
When using the Effect Hook, passing a dependency array as the second argument for useEffect() is the best way to determine when our effect is and is not called.

Remove the if (!categories) condition, and pass an empty dependency array so that this effect is only called after the first render.
  */
  useEffect(() => {
      get('/categories').then((response) => {
        setCategories(response.data);
      });
    }, []);
  
  // list the 2 variables that this effect depends on in the dependency array
  useEffect(() => {
    if (selectedCategory && !items[selectedCategory]) {
      // fetch data and store it to local state
      get(`/items?category=${selectedCategory}`).then((response) => {
        setItems((prev) => (
          { ...prev, [selectedCategory]: response.data }));
        });
}
}, [items, selectedCategory]);


  if (!categories) {
    return <p>Loading..</p>;
  }

  return (
    <div className='App'>
      <h1>Clothes 'n Things</h1>
      <nav>
        {categories.map((category) => (
          <button key={category} onClick={() => setSelectedCategory(category)}>
            {category}
          </button>
        ))}
      </nav>
      <h2>{selectedCategory}</h2>
      <ul>
        {!items[selectedCategory]
          ? null
          : items[selectedCategory].map((item) => <li key={item}>{item}</li>)}
      </ul>
    </div>
  );
}

##### 